In [1]:
!pip install transformers
!pip install pandas scikit-learn torch



  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')


In [6]:
#Load and Balance the Dataset:
df = pd.read_csv('/content/drive/MyDrive/MINI PROJECT/DATASET/augmented_dataset1.csv')
print(df.tail())

df = pd.concat([
    df[df['Label'] == 1].sample(n=300, random_state=42),
    df[df['Label'] == 0].sample(n=300, random_state=42),
    df[df['Label'] == 2].sample(n=300, random_state=42)
])

df = df.reset_index(drop=True)
print(df.head())


         PID                                          Text data  Label
18052  18052  ov trudging remembe dehydrated pregame adays d...      2
18053  18053  exacerbate floated prescribes wonky combo fibr...      2
18054  18054  voice heavy terrible awake help need night its...      2
18055  18055  wonder doesn smile face hate what right can be...      2
18056  18056  redacted overshare detail center colleges room...      2
    PID                                          Text data  Label
0  2868  my to have about being in not any way and im i...      1
1  5924  for to the know but that in few of broke up wa...      1
2  3764  new being same dont this shit will so me think...      1
3  4144  for my to have once the new year there anymore...      1
4  2780  for my to have the new year anymore about but ...      1


In [7]:
#Initialize BERT Model and Tokenizer:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [8]:
#Define a Function to Extract BERT Features:
def get_bert_features(text_batch, model, tokenizer):
    encoded_inputs = tokenizer.batch_encode_plus(
        text_batch,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    input_ids = encoded_inputs['input_ids']
    attention_mask = encoded_inputs['attention_mask']

    with torch.no_grad():
        last_hidden_states = model(input_ids, attention_mask=attention_mask)

    features = last_hidden_states[0][:, 0, :].numpy()
    return features


In [9]:
#Process Data in Batches:
batch_size = 100
features_list = []
for i in range(0, df.shape[0], batch_size):
    text_batch = df['Text data'].iloc[i:i+batch_size].tolist()
    batch_features = get_bert_features(text_batch, model, tokenizer)
    features_list.append(batch_features)

features = np.concatenate(features_list, axis=0)
labels = df['Label'].values


In [10]:
#Split Data into Training and Testing Sets:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)


In [12]:
#Train and Evaluate a Random Forest Classifier:
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(x_train, y_train)

pred = rf_clf.predict(x_test)
print(classification_report(y_test, pred))


              precision    recall  f1-score   support

           0       0.76      0.63      0.69        67
           1       0.68      0.83      0.75        60
           2       0.78      0.75      0.77        53

    accuracy                           0.73       180
   macro avg       0.74      0.74      0.73       180
weighted avg       0.74      0.73      0.73       180



In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
